# Prompt Optimisation Using MIPRO

This method, MIPRO, rewrites the prompt. So start with a simple vague prompt and get it re-written to something that fits. 

Use Case:  
Create Meal/Dish classes with a prompt that is vague to start off with (i.e. "Create a meal"). Then put in a constraint that it only knows about during the optimisation phase (i.e. "Only select Fish dishes"). Then optimise the prompt and see if it changes it to cater for this constraint.

# Setup
Note: Update OPENAI_API_KEY in env settings or MySettings class, see repo's README.

In [14]:
import sys
import os
import dspy 
from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

Getting keys from environment variables


# Create Domain

In [ ]:
# Create domain classes

from pydantic import BaseModel
from typing import Literal

class Dish(BaseModel):
    # Output
    dish_type: Literal["Fish", "Vegetarian", "Chicken"] = dspy.OutputField() # This field is locked to a range of the Literal
    name: str = dspy.OutputField()
    
class Meal(dspy.Signature):
    
    dish_suggestion = dspy.InputField("Suggest a meal for the given day of the week.")

    # Outputs
    dish: Dish = dspy.OutputField()
    day_of_the_week: Literal["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"] = dspy.OutputField()

#class MealGenerator(dspy.Signature):
    

# class MealGenerator(dspy.Module):
#     def __init__(self):
#         self.prompt = dspy.Predict(Meal)

#     def forward(self):
#         return self.prompt()

# Create Trainset
The ideal number of trainset examples for the DSPy MIPRO Prompt Optimizer depends on the complexity of your task and the variability in your data. However, here are some general guidelines:

Recommended Starting Point
1. Small-scale tasks or prototyping: Start with **20–50 examples**.
1. Medium complexity tasks: Use **100–500 examples**.
1. High variability or complex tasks: Consider **1,000+ examples**, if computationally feasible.

In [16]:
# Creat trainset with 25 examples. 
# Tip: Use ChatGPT to generate these examples for you.

trainset = [
    {'preferred_day': 'Tuesday', 'dish': {'dish_type': 'Chicken', 'name': 'Chicken Tikka Masala'}, 'day_of_the_week': 'Tuesday'},
    {'preferred_day': 'Monday', 'dish': {'dish_type': 'Vegetarian', 'name': 'Caprese Salad'}, 'day_of_the_week': 'Monday'},
    {'preferred_day': 'Monday', 'dish': {'dish_type': 'Fish', 'name': 'Baked Cod'}, 'day_of_the_week': 'Monday'},
    {'preferred_day': 'Friday', 'dish': {'dish_type': 'Chicken', 'name': 'Chicken Alfredo'}, 'day_of_the_week': 'Friday'},
    {'preferred_day': 'Sunday', 'dish': {'dish_type': 'Vegetarian', 'name': 'Chickpea Curry'}, 'day_of_the_week': 'Sunday'},
    {'preferred_day': 'Thursday', 'dish': {'dish_type': 'Chicken', 'name': 'Chicken Tikka Masala'}, 'day_of_the_week': 'Thursday'},
    {'preferred_day': 'Monday', 'dish': {'dish_type': 'Vegetarian', 'name': 'Caprese Salad'}, 'day_of_the_week': 'Monday'},
    {'preferred_day': 'Friday', 'dish': {'dish_type': 'Chicken', 'name': 'Grilled Chicken Salad'}, 'day_of_the_week': 'Friday'},
    {'preferred_day': 'Saturday', 'dish': {'dish_type': 'Vegetarian', 'name': 'Vegetable Stir Fry'}, 'day_of_the_week': 'Saturday'},
    {'preferred_day': 'Monday', 'dish': {'dish_type': 'Fish', 'name': 'Baked Cod'}, 'day_of_the_week': 'Monday'},
    {'preferred_day': 'Thursday', 'dish': {'dish_type': 'Vegetarian', 'name': 'Lentil Soup'}, 'day_of_the_week': 'Thursday'},
    {'preferred_day': 'Friday', 'dish': {'dish_type': 'Fish', 'name': 'Fish Tacos'}, 'day_of_the_week': 'Friday'},
    {'preferred_day': 'Wednesday', 'dish': {'dish_type': 'Vegetarian', 'name': 'Vegetable Stir Fry'}, 'day_of_the_week': 'Wednesday'},
    {'preferred_day': 'Tuesday', 'dish': {'dish_type': 'Fish', 'name': 'Baked Cod'}, 'day_of_the_week': 'Tuesday'},
    {'preferred_day': 'Saturday', 'dish': {'dish_type': 'Vegetarian', 'name': 'Chickpea Curry'}, 'day_of_the_week': 'Saturday'},
    {'preferred_day': 'Tuesday', 'dish': {'dish_type': 'Chicken', 'name': 'Chicken Noodle Soup'}, 'day_of_the_week': 'Tuesday'},
    {'preferred_day': 'Wednesday', 'dish': {'dish_type': 'Vegetarian', 'name': 'Mushroom Risotto'}, 'day_of_the_week': 'Wednesday'},
    {'preferred_day': 'Sunday', 'dish': {'dish_type': 'Chicken', 'name': 'Grilled Chicken Salad'}, 'day_of_the_week': 'Sunday'},
    {'preferred_day': 'Wednesday', 'dish': {'dish_type': 'Chicken', 'name': 'BBQ Chicken'}, 'day_of_the_week': 'Wednesday'},
    {'preferred_day': 'Sunday', 'dish': {'dish_type': 'Chicken', 'name': 'Grilled Chicken Salad'}, 'day_of_the_week': 'Sunday'},
    {'preferred_day': 'Thursday', 'dish': {'dish_type': 'Chicken', 'name': 'Chicken Tikka Masala'}, 'day_of_the_week': 'Thursday'},
    {'preferred_day': 'Wednesday', 'dish': {'dish_type': 'Fish', 'name': 'Baked Cod'}, 'day_of_the_week': 'Wednesday'},
    {'preferred_day': 'Wednesday', 'dish': {'dish_type': 'Fish', 'name': 'Grilled Salmon'}, 'day_of_the_week': 'Wednesday'},
    {'preferred_day': 'Thursday', 'dish': {'dish_type': 'Chicken', 'name': 'BBQ Chicken'}, 'day_of_the_week': 'Thursday'},
    {'preferred_day': 'Wednesday', 'dish': {'dish_type': 'Chicken', 'name': 'BBQ Chicken'}, 'day_of_the_week': 'Wednesday'}
]

# Creat LLM Clients

In [17]:
# Smaller LLM, this is the one that we are trying to optimize for, the prompts are going to be tweaked
# to get the best out of this model
lm_gpt35 = dspy.LM('gpt-3.5-turbo', model_type='chat', cache=False, api_key=os.getenv("OPENAI_API_KEY"))
#dspy.configure(lm=lm_gpt35)

# # Larger LLM, this is the one that we are going to use to optimize the prompts
# # It will be the helper/teach/AI Judge to assist in the optimization process
lm_gpt4 = dspy.LM('gpt-4.1', model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
dspy.configure(lm=lm_gpt4)

# Optimization

Purpose of metric in MIPRO
The metric function in MIPRO is used to evaluate the quality of a model’s output against the expected result. It should return a boolean or score indicating whether the output is correct.

In [23]:
def validate_match(expected: Meal, actual: Meal) -> bool:
    return (expected.dish.dish_type == "Fish")

matcher = dspy.Predict(Meal)
md(matcher(Meal))


IndexError: list index out of range

In [ ]:
from dspy.teleprompt import *


tp = dspy.MIPROv2(metric=validate_match, auto="light", prompt_model=lm_gpt35, task_model=lm_gpt4)
optimized_matcher = tp.compile(matcher, trainset=trainset, requires_permission_to_run=False)